In [1]:
import rootutils

rootutils.setup_root("./", indicator=".project-root", pythonpath=True)

%load_ext autoreload
%autoreload 2

import torch
import torch_geometric
from topobenchmarkx.data.datasets import CustomDataset
import hydra
from hydra import initialize, compose
from topobenchmarkx.data.dataloader_fullbatch import FullBatchDataModule
from topobenchmarkx.data.load.loaders import (
    GraphLoader,
    SimplicialLoader,
    HypergraphLoader,
)
from toponetx.classes import CellComplex
import networkx as nx
from topomodelx.utils.sparse import from_sparse

initialize(config_path="../configs", job_name="job")
config = compose(config_name="train.yaml", return_hydra_config=True)

/var/folders/95/10br05cj3zs5lc7k18dry7dh0000gn/T/ipykernel_5047/3520126482.py:23: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="../configs", job_name="job")


In [2]:
graph_loader = hydra.utils.instantiate(config.dataset, _recursive_=False)
data = graph_loader.load()[0].data_lst[0]

Transform parameters are the same, using existing data_dir: /Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/datasets/graph/TUDataset/MUTAG/Identity/3446711709


In [10]:
def get_complex_connectivity(complex, max_rank):
    connectivity = {}
    connectivity.update(
        {
            "incidence_{}".format(rank_idx): from_sparse(
                complex.incidence_matrix(rank_idx)
            )
            for rank_idx in range(1, max_rank)
        }
    )
    for rank_idx in range(max_rank + 1):
        try:
            down_laplacian = from_sparse(complex.down_laplacian_matrix(rank=rank_idx))
        except ValueError:
            down_laplacian = torch.zeros(
                (complex.shape[rank_idx], complex.shape[rank_idx])
            ).to_sparse()
        # Up laplacian
        try:
            up_laplacian = from_sparse(complex.up_laplacian_matrix(rank=rank_idx))
        except ValueError:
            up_laplacian = torch.zeros(
                (complex.shape[rank_idx], complex.shape[rank_idx])
            ).to_sparse()
        hodge_laplacian = down_laplacian + up_laplacian
        connectivity.update({"down_laplacian_{}".format(rank_idx): down_laplacian})
        connectivity.update({"up_laplacian_{}".format(rank_idx): up_laplacian})
        connectivity.update({"hodge_laplacian_{}".format(rank_idx): hodge_laplacian})
    connectivity.update({"shape": complex.shape})
    return connectivity

In [11]:
max_rank = 2
n_nodes = data.x.shape[0]
# edge_index = torch_geometric.utils.to_undirected(data.edge_index)
edges = [(i.item(), j.item()) for i, j in zip(data.edge_index[0], data.edge_index[1])]
G = nx.Graph()
G.add_edges_from(edges)
cycles = nx.cycle_basis(G)
cell_complex = CellComplex(G)
cell_complex.add_cells_from(cycles, rank=max_rank)
lifted_topology = get_complex_connectivity(cell_complex, max_rank)

In [12]:
lifted_topology

{'incidence_1': tensor(indices=tensor([[ 0,  0,  1,  1,  2,  2,  2,  3,  3,  3,  4,  4,  5,  5,
                          6,  6,  7,  7,  8,  8,  8,  9,  9, 10, 10, 10, 11, 12],
                        [ 0,  1,  0,  2,  2,  3,  4,  3,  5,  6,  5,  7,  7,  8,
                          8,  9,  4,  9,  6, 10, 11,  1, 10, 11, 12, 13, 12, 13]]),
        values=tensor([-1., -1.,  1., -1.,  1., -1., -1.,  1., -1., -1.,  1.,
                       -1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1.,
                        1.,  1., -1., -1.,  1.,  1.]),
        size=(13, 14), nnz=28, layout=torch.sparse_coo),
 'down_laplacian_0': tensor(indices=tensor([], size=(2, 0)),
        values=tensor([], size=(0,)),
        size=(13, 13), nnz=0, layout=torch.sparse_coo),
 'up_laplacian_0': tensor(indices=tensor([[ 0,  0,  0,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,
                          4,  4,  4,  5,  5,  5,  6,  6,  6,  7,  7,  7,  8,  8,
                          8,  8,  9,  9,  9, 10, 10, 1

In [16]:
cell_complex = CellComplex(G)
cell_complex.add_cells_from(cycles, rank=2)

In [19]:
cell_complex.shape

(13, 14, 2)